In [30]:
import os
import re
import json
import io
import requests
import ast
from typing import Optional
from datetime import date

from dotenv import load_dotenv

from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_text_splitters import RecursiveCharacterTextSplitter
from unstructured.partition.html import partition_html

In [31]:
test_url = "https://www.nordstrom.com/browse/men/shoes/boots"

In [32]:
def get_test_html() -> str:
    with open("test_web_page.html", "r") as f:
        return f.read()

In [33]:
# response = requests.get(test_url)
# response.raise_for_status()
# print(f"Response: {response}")
# html_content = response.text
# print(f"HTML content: {html_content}")
# Convert text to bytes using utf-8 encoding
# html_content_bytes = io.BytesIO(html_content.encode('utf-8'))
# extracted_elements = partition_html(text=html_content)

In [34]:
# print([element.text for element in extracted_elements])

[]


In [ ]:
# TODO: Issue with partition_html - doesn't know what elements are associated with each other
# Debatable whether it's better to use the custom parser system or use partition_html out of the box

In [ ]:
def split_html(html: str) -> list[str]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=5000,
        chunk_overlap=100,
        length_function=len,
        is_separator_regex=False,
    )
    return text_splitter.split_text(html)

In [59]:
test_file = get_test_html()
elements = partition_html(text=test_file)
print(f"len(elements): {len(elements)}")
print([element.text for element in elements[420:500]])
print([element.metadata.link_urls for element in elements[420:500]])


len(elements): 649
['New In: Men', "Explore new in men's ready-to-wear collection and new arrivals in men's shoes.", 'Lasered denim jacket', '$ 2,550', 'Shop This', "Men's Gucci Re-Web sneaker", '$ 1,150', 'Shop This', 'Tapered denim pant with Web', '$ 1,300', 'Shop This', 'Rib knit wool hat', '$ 480', 'Shop This', 'G-Timeless watch, 38mm', '$ 1,550', 'Shop This', 'Square frame sunglasses', '$ 480', 'Shop This', 'Shop This', 'Wool sweater with Gucci intarsia', '$ 1,250', 'Shop This', 'VIRTUAL TRY-ON', "Men's Horsebit 1953 loafer", '$ 990', 'Shop This', 'Polyester GG jacquard pant with Web', '$ 1,750', 'Shop This', 'Icon 18k GG thin ring', '$ 1,850', 'Shop This', 'RUNWAY', 'Jackie large shoulder bag', '$ 4,800', 'Shop This', 'GG Marmont chain bracelet', '$ 340', 'Shop This', 'Rib knit wool hat', '$ 480', 'Shop This', 'Denim shirt with lasered Gucci detail', '$ 2,650', 'Shop This', "Men's ankle boot with Horsebit", '$ 1,270', 'Shop This', 'RUNWAY', 'GG Marmont thin belt', '$ 530', 'Shop 